In [7]:
import pandas as pd

def load_stock_data(file_name):
    df = pd.read_pickle(file_name)
    return df

def make_dataset(df, time_lags=5):
    df_lag = pd.DataFrame(index=df.index)
    df_lag['Close'] = df['Close']
    df_lag['Volume'] = df['Volume']
    
    df_lag['Close_Lag%s' % str(time_lags)] = df['Close'].shift(time_lags)
    df_lag['Close_Lag%s_Change' % str(time_lags)] = df_lag['Close_Lag%s' % str(time_lags)].pct_change()*100.0
    
    df_lag['Volume_Lag%s' % str(time_lags)] = df['Volume'].shift(time_lags)
    df_lag['Volume_Lag%s_Change' % str(time_lags)] = df_lag['Volume_Lag%s' % str(time_lags)].pct_change()*100.0
    
    df_lag['Close_Direction'] = np.sign(df_lag['Close_Lag%s_Change' % str(time_lags)])
    df_lag['Volume_Direction'] = np.sign(df_lag['Volume_Lag%s_Change' % str(time_lags)])
    
    return df_lag.dropna(how='any')


def split_dataset(df, input_column_array, output_column, split_ratio):
    split_date = get_date_by_percent(df.index[0], df.index[df.shape[0]-1], split_ratio)
    
    input_data = df[input_column_array]
    output_data = df[output_column]
    
    X_train = input_data[input_data.index < split_date]
    X_test = input_data[input_data.index >= split_date]
    Y_train = output_data[output_data.index < split_date]
    Y_test = output_data[output_data.index >= split_date]
    
    return X_train, X_test, Y_train, Y_test


import numpy as np
import datetime

def get_date_by_percent(start_date, end_date, percent):
    days = (end_date - start_date).days
    target_days = np.trunc(days * percent)
    target_date = start_date + datetime.timedelta(days=target_days)
    return target_date


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

def do_logistic_regression(x_train, y_train):
    classifier = LogisticRegression()
    classifier.fit(x_train, y_train)
    return classifier


def do_random_forest(x_train, y_train):
    classifier = RandomForestClassifier()
    classifier.fit(x_train, y_train)
    return classifier


def do_svm(x_train, y_train):
    classifier = SVC()
    classifier.fit(x_train, y_train)
    return classifier


def test_classifier(classifier, x_test, y_test):
    pred = classifier.predict(x_test)
    
    hit_count = 0
    total_count = len(y_test)
    for index in range(total_count):
        if pred[index] == y_test[index]:
            hit_count = hit_count + 1
            
    hit_ratio = hit_count / total_count
    score = classifier.score(x_test, y_test)
    
    return hit_ratio, score


if __name__ == '__main__':
    for time_lags in range(1, 6):
        print('- Time Lags=%s' % (time_lags))
        
        for company in ['samsung', 'hanmi']:
            df_company = load_stock_data('%s.data'%(company))
            
            df_dataset = make_dataset(df_company, time_lags)
            X_train, X_test, Y_train, Y_test = split_dataset(df_dataset, ['Close_Lag%s' % (time_lags)], 'Close_Direction', 0.75)
            
            lr_classifier = do_logistic_regression(X_train, Y_train)
            lr_hit_ratio, lr_score = test_classifier(lr_classifier, X_test, Y_test)
            
            rf_classifier = do_random_forest(X_train, Y_train)
            rf_hit_ratio, rf_score = test_classifier(rf_classifier, X_test, Y_test)
            
            svm_classifier = do_svm(X_train, Y_train)
            svm_hit_ratio, svm_score = test_classifier(svm_classifier, X_test, Y_test)
            
            print('%s : Hit Ratio - Logistic Regression=%0.2f, RandomForest=%0.2f, SVM=%0.2f' % (company, lr_hit_ratio, rf_hit_ratio, svm_hit_ratio))
    

- Time Lags=1
samsung : Hit Ratio - Logistic Regression=0.54, RandomForest=0.55, SVM=0.50
hanmi : Hit Ratio - Logistic Regression=0.57, RandomForest=0.41, SVM=0.57
- Time Lags=2
samsung : Hit Ratio - Logistic Regression=0.54, RandomForest=0.52, SVM=0.50
hanmi : Hit Ratio - Logistic Regression=0.57, RandomForest=0.39, SVM=0.59
- Time Lags=3
samsung : Hit Ratio - Logistic Regression=0.54, RandomForest=0.52, SVM=0.50
hanmi : Hit Ratio - Logistic Regression=0.57, RandomForest=0.36, SVM=0.59

C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\env


- Time Lags=4
samsung : Hit Ratio - Logistic Regression=0.55, RandomForest=0.52, SVM=0.50
hanmi : Hit Ratio - Logistic Regression=0.55, RandomForest=0.38, SVM=0.57
- Time Lags=5
samsung : Hit Ratio - Logistic Regression=0.55, RandomForest=0.53, SVM=0.49
hanmi : Hit Ratio - Logistic Regression=0.56, RandomForest=0.44, SVM=0.58


C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:18: RuntimeWarning: invalid value encountered in sign
C:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:19: RuntimeWarning: invalid value encountered in sign
